In [1]:
import pandas as pd
import geopandas as gpd

In [76]:
#Load Datasets

## load Project Sunroof Data
sr_data = pd.read_csv("data/project-sunroof-census_tract.csv", index_col="region_name")
sr_data = sr_data.rename_axis(index={"region_name": "GEOID"})
sr_data.index = sr_data.index.astype(int)
#print(sr_data.head(5))

#load Energy Equity Project Data
eep_data = pd.read_csv("data/eep_final_data.csv", index_col="GEOID")
eep_data.index = eep_data.index.astype(int)
#print(eep_data.head(5))

#load Justice40 Data
j40_data = pd.read_csv("data/1.0-communities.csv", index_col="Census tract 2010 ID")
j40_data = j40_data.rename_axis(index={"Census tract 2010 ID": "GEOID"})
j40_data.index = j40_data.index.astype(int)
#print(j40_data.head(5))

#load Opportunity Zone Data
oz_data = pd.read_csv("data/Opportunity_Zone_ExportTable1.csv", index_col="Tract_Boundaries_ID")
oz_data = oz_data.rename_axis(index={"Tract_Boundaries_ID": "GEOID"})
oz_data.index = oz_data.index.astype(int)
#print(oz_data.head(5))

#load REPLICA data
replica_data = pd.read_csv("data/seeds_ii_replica.csv", index_col="geoid")
replica_data = replica_data.rename_axis(index={"geoid": "GEOID"})
replica_data.index = replica_data.index.astype(int)
#print(oz_data.head(5))

/var/folders/67/r87g720s2653g554zdhtz1vh0000gn/T/ipykernel_44423/3391262701.py:15: DtypeWarning: Columns (18,26,70,72,85,131) have mixed types. Specify dtype option on import or set low_memory=False.
  j40_data = pd.read_csv("data/1.0-communities.csv", index_col="Census tract 2010 ID")
/var/folders/67/r87g720s2653g554zdhtz1vh0000gn/T/ipykernel_44423/3391262701.py:27: DtypeWarning: Columns (148) have mixed types. Specify dtype option on import or set low_memory=False.
  replica_data = pd.read_csv("data/seeds_ii_replica.csv", index_col="geoid")


In [81]:
#Load Geospatial Dataset

#load urban areas dataset from US Census Bureau
urban_census_geodata = gpd.read_file("data/tl_2010_us_uac10.shx")

#load Energy Equity Project geojson for geometry
eep_geodata = gpd.read_file("data/eep_final_simplified.json")

In [84]:
#Extract GEOID and Geometry to join into other datasets later
tracts_geoid = eep_geodata[["GEOID", "geometry"]]
tracts_geoid = tracts_geoid.set_index("GEOID")
tracts_geoid.index = tract_geoid.index.astype(int)

In [12]:
#Check datasets for shape
print(sr_data.shape)
print(eep_data.shape)
print(j40_data.shape)

(56940, 30)
(83548, 43)
(74134, 132)


In [77]:
# Create Category 1 Dataset for Main Criteria 
# based on 45D(e) Internal Revenue Service Guidelines 
# which defines "low-income community" census tracts where: 
# 1) Poverty rate > 20% OR 
# 2) Median family income <= 80% of statewide MFI

# Create a new column 'qualified' and initialize it with False
oz_data['qualified'] = False

columns_to_convert = ['Poverty_csv_Percent_below_pover', 'MedianFamilyIncome_csv_Median_F', 'State_MedFamInc', 'Metro_MedFamInc']

# Convert selected columns to float, ignoring non-numeric values
oz_data[columns_to_convert] = oz_data[columns_to_convert].apply(pd.to_numeric, errors='coerce')


# Set 'qualified' if poverty percentage is >= 20
oz_data.loc[oz_data['Poverty_csv_Percent_below_pover'] >= 20, 'qualified'] = True

# Set 'qualified' if tract is in metro area and median family income does not exceed 
# the greater of 80% statewide median family income and 80% metropolitan area median family income
oz_data.loc[(oz_data['Tract_by_Metro_Area_csv_class'] == "Metro") & 
             (oz_data['MedianFamilyIncome_csv_Median_F'] < oz_data[['State_MedFamInc', 'Metro_MedFamInc']].max(axis=1)), 'qualified'] = True

# Set 'qualified' if tract not in a metro area and median family income does not exceed 80% statewide median family income
oz_data.loc[(oz_data['Tract_by_Metro_Area_csv_class'] != "Metro") & 
             (oz_data['MedianFamilyIncome_csv_Median_F'] < 0.8*oz_data['State_MedFamInc']), 'qualified'] = True

oz_data_qualified = oz_data.loc[oz_data["qualified"] == True][['qualified']]


In [78]:
#Merge datasets
sr_selected_cols = sr_data[['percent_covered', 'number_of_panels_total', 'number_of_panels_median', 'kw_median', 'kw_total', 
                            'yearly_sunlight_kwh_median', 'yearly_sunlight_kwh_total', 'carbon_offset_metric_tons', 'existing_installs_count']]

eep_selected_cols = eep_data[['STATE_NAME', 'energy_burden', 'num_solar_installers', 'median_income_solar', 'total_pop', 'year_built', 
                              'total_households', 'median_income', 'owner_occupied', 'renter_occupied', 'bipoc_percent', 'community_power_score',  ]]

j40_selected_cols = j40_data[['Total population', 'Percent of individuals < 100% Federal Poverty Line (percentile)', 'PM2.5 in the air (percentile)', 
                              'Current asthma among adults aged greater than or equal to 18 years (percentile)', 'Housing burden (percent) (percentile)', 
                              'Unemployment (percent) (percentile)']]

merged_data = pd.merge(sr_selected_cols, eep_selected_cols, left_index=True, right_index=True)
merged_data = pd.merge(merged_data, j40_selected_cols, left_index=True, right_index=True )
merged_data = pd.merge(merged_data, oz_data_qualified, left_index=True, right_index=True )
#does that do the filtering properly? ^^^

            percent_covered  number_of_panels_total  number_of_panels_median   
GEOID                                                                          
1001020200       104.472843                 59054.0                     44.0  \
1001020300       101.278409                 80364.0                     41.0   
1001020700       116.604128                104519.0                     46.0   
1003010400        38.358459                106874.0                     51.0   
1003010903       102.763819                169232.0                     56.0   

            kw_median  kw_total  yearly_sunlight_kwh_median   
GEOID                                                         
1001020200      11.00  14763.50                 13564.14519  \
1001020300      10.25  20091.00                 12725.93064   
1001020700      11.50  26129.75                 14215.97524   
1003010400      12.75  26718.50                 16556.98295   
1003010903      14.00  42308.00                 18190.67319  

In [80]:
#Clean Data
cleaned_data = merged_data.copy()

# Count the initial number of data points
initial_count = len(cleaned_data)

# Create a boolean mask to filter out 
# 1) energy_burden data points at or above 100%
energy_burden_mask = (cleaned_data['energy_burden'] < 100) & (cleaned_data['energy_burden'] >= 0)

# Filter the DataFrame using the boolean masks while counting number of removed data points
cleaned_data = cleaned_data[energy_burden_mask]
new_count = len(cleaned_data)
removed_count = initial_count - new_count
print("Number of data points removed for energy burden:", removed_count)

Number of data points removed for energy burden: 49


In [85]:
#Write merged and cleaned dataset for further analysis
cleaned_data.to_csv('cleaned_data.csv', index=True)

#write tract_geoid to file
tracts_geoid.to_file("tracts.geojson", driver='GeoJSON')